In [ ]:
import numpy as np
import processingIO as pio
import matplotlib.pyplot as plt
import pandas as pd

from pathlib import Path
from dataUtilities import filterData
from forceClass import Forces, ForceCoefficients

from scipy import stats
from scipy.optimize import curve_fit
from scipy.stats.distributions import  t

from mpl_toolkits.axes_grid1.inset_locator import (inset_axes, InsetPosition,
                                                  mark_inset)

In [ ]:
def power_law(xData, a, b, c):
        return a + (b * xData) + (c * xData**2)

def fit_data(xData, yData):

    popt, pcov = curve_fit(power_law, xData, yData, p0=[0.5, 0.5, 0.5])

    residuals = yData - power_law(xData, *popt)

    ss_res = np.sum(residuals**2)
    ss_total = np.sum((yData-np.mean(yData))**2)

    r_squared = 1 - (ss_res/ss_total)

    return popt, r_squared

In [ ]:
data_location = r'/location/of/data'

force_paths = pio.get_files(data_location, 'force.dat')
force_paths.sort()
coefficients_paths = pio.get_files(data_location, 'coefficient.dat')
coefficients_paths.sort()
solver_paths = pio.get_files(data_location, 'log.solver')
solver_paths.sort()

tmp = []

constantDict = {'parentCase':[], 'specificCase':[], 'totalTime':[]}

for solverFile in solver_paths:
    constantDict['parentCase'].append(solverFile.parts[-4])
    constantDict['specificCase'].append(solverFile.parts[-3])
    with open(solverFile, 'r') as f:
        for line in f:
            if 'ExecutionTime' in line:
                tmp.append(line.split()[2])

    constantDict['totalTime'].append(tmp[-1]) 

cycles =  2
total_cycles = 4.0

forces = [Forces(filepath, cycles, total_cycles, True, True) for filepath in force_paths]
coeffs = [ForceCoefficients(filepath, cycles, total_cycles, True, True) for filepath in coefficients_paths]

In [ ]:
plt.rcParams.update({'font.size':20})

density = 1025
fudgeFactor = 1.0
U = 1.0
kinvis = 2.5e-5
chord = 1.0

St = [0.05, 0.10, 0.25, 0.50, 0.75, 1.0, 1.25, 1.50, 2.50]
average_fx = []
case = []

fig1, ax1 = plt.subplots(figsize=(12,10))
for force in forces:
    average_fx.append(force.averageFilteredForces['total']['x']/density)
    case.append(force.specific_case)

    ax1.plot(force.filteredForces['time']/force.filteredForces['time'][-1], force.filteredForces['total']['x'], label=force.specific_case)

ax1.set_xlabel('Time [s]')
ax1.set_ylabel('$\mathrm{F_x}$ [N]')
#ax2 = plt.axes([0, 0, 1, 1])
#ip = InsetPosition(ax1, [0.2,0.4,0.55,0.55])
#ax2.set_axes_locator(ip)
#mark_inset(ax1, ax2, loc1=2, loc2=4, fc="none", ec='0.5')

ax1.set_ylim([-5000, 5000])
ax1.legend()

average_fx = np.asarray(average_fx)

In [ ]:
anderson_data = r'/location/of/data'
andersonDF = pd.read_excel(anderson_data, sheet_name='alpha15')

In [ ]:
e_force = np.array([0.0703, 0.08278, -0.00388, -0.188966, -0.56408, -1.14173, -1.8766147, -2.850414246, -8.68766])
e_st = np.array([0.05, 0.1, 0.25, 0.5, 0.75, 1, 1.25, 1.5, 2.5])

e_fit, e_r_squared = fit_data(e_st, e_force)
ika_fit, ika_r_squared = fit_data(e_st, average_fx)

fig2, ax2 = plt.subplots(figsize=(12,10))
ax2.plot(e_st, e_force, 'o-', label="Experiment")
ax2.plot(e_st, power_law(e_st, *e_fit), '--', label='Experiment Fit Line')
ax2.plot(e_st, average_fx, 'o-', label='Ika Flow')
ax2.plot(e_st, power_law(e_st, *ika_fit), '--', label='Ika Fit Line')

ax2.set_xlabel(r'St')
ax2.set_ylabel('$\mathrm{F_x}$')

ax2.legend()